In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv
from utils import *


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import random
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os


default_or_options = {
    "max_tokens": 8191,

    "extra_body": {
        "provider": {
            "ignore": [
                "Hyperbolic"
            ],
            "require_parameters": True,
        },
    }
}
domain_events = load_dataset("lucaelin/generic_domain_events_v1", split="train")


def generate_event_dialog(entry):
    results = []

    available_models = [
        #'anthropic/claude-3.5-sonnet-20240620',
        #'openai/chatgpt-4o-latest',
        #'openai/gpt-4o-mini-2024-07-18',
        #'meta-llama/llama-3.1-405b-instruct',
        #'meta-llama/llama-3.1-70b-instruct',
        #'meta-llama/llama-3.1-70b-instruct',
        "mistralai/mistral-nemo",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        #"mistralai/mixtral-8x22b-instruct",
        #"mistralai/mistral-small",
        "mistralai/mistral-large"
    ]

    related_entries = domain_events.filter(lambda x: x["profession"] == entry["profession"] and x["process"] == entry["process"]).to_list()


    model = random.choice(available_models)

    select_event_name = [{
        "type": "function",
        "function": {
            "name": "select_event_name",
            "description": "Select an event name that continues the currently ongoing process.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_name": {
                        "type": "string",
                        "enum": [event["name"].strip() for event in related_entries],
                    },
                },
                "required": ["event_name"],
            },
        },
    }]

    generate_event_functions = [
        {
            "type": "function",
            "function": {
                "name": f"generate_{event["name"].replace(' ','_').lower()}_event",
                "description": event["name"].strip()+'\n'+event["description"],
                "parameters": json.loads(event['schema']),
            },
        } for event in related_entries
    ]

    events = []

    while len(events) < random.randint(2, 6):
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"You roleplay an expert {entry['profession']} currently in the process of {entry['process']}. Be creative during this roleplay, you are in the lead, so you need to fill the story with detail.\n"+
                    "Your task is to note down the progress made. You should be creative and fill the story with detail."
            }, {
                "role": "user",
                "content": entry['situation']+'\n'+
                    f"Please select one of the available events to describes the currently ongoing process of {entry['process']}. Use the select_event_name function to select an event name.",
            }]+ ([{
                "role": "assistant",
                "content": '\n'.join([event["name"] for event in events]),
            }, {
                "role": "user",
                "content": f"Please choose a different event to continue the current process. Use the select_event_name function to select an event name.",
            }] if events else []),
            tools=select_event_name,
            tool_choice="required",
            **default_or_options,
        )
        print(openrouter_response)
        event_name_selection = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
        event_name = event_name_selection["event_name"].strip()
        print('selected', event_name)

        selected_event_functions = [function for function in generate_event_functions if function["function"]["description"].split('\n')[0].strip() == event_name]
        print('available functions', [function["function"]["description"].split('\n')[0].strip() for function in generate_event_functions])
        print('selected function', selected_event_functions)

        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"You roleplay an expert {entry['profession']} currently in the process of {entry['process']}. Be creative during this roleplay, you are in the lead, so you need to fill the story with detail.\n"+
                    "Your task is to note down the progress made. You should be creative and fill the story with detail."
            }, {
                "role": "user",
                "content": entry['situation']+'\n'+
                    f"Please select one of the available events to describes the currently ongoing process of {entry['process']}. Use the select_event_name function to select an event name.",
            }]+ ([{
                "role": "assistant",
                "content": '\n'.join([json.dumps(event) for event in events]),
            }, {
                "role": "user",
                "content": f"Please continue this process using the {event_name} event. Use the generate_event function to generate the event.",
            }] if events else []),
            tools=selected_event_functions,
            tool_choice="required",
            **default_or_options,
        )
        event = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
        print(event_name, ':', event)

        #print(request)
        events.append({"name":event_name, 'content': event})

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are the digital Assistant of an expert {entry['profession']} currently accompanying the process of {entry['process']}.\n"+
                "Your task is to provide situational commentary on the events that have unfolded.\n"+
                "Here is an example styles from a Dentist roleplay that you should follow:\n"+
                " - 'Initial findings on Adam show 3 cavities. He has a toothache and is in pain.'\n"+
                " - 'Preparing X-Ray on Thomas Delvenstein while 3 more patiens are in the waiting room.'\n"+
                " - 'The second room is cleaned and prepared. The procedure in the first is still on-going.'\n"+
                f"Your responses should be informative to the expert {entry['profession']} as they are the one leading the process. Do not mention the protagonist by name, refer to them as 'you'. The last event is the most recent one that occured within the last new seconds and should be regarded as the present state.\n"+
                f"You should be creative, but strictly stick to the information available in the event, only summarize the sequence of events that have unfolded into a single sentence. You should keep your commentary concise. Do not assume what {entry['profession']} has done, only on the events that actually happend as a result. Focus on the outcome of the events, not the process itself.",
        }, {
            "role": "user",
            "content": entry['situation']+'\n'+
                f"Please provide a single sentence commentary on these events:\n"+
                '\n'.join([json.dumps(event) for event in events]),
        }],
        **default_or_options,
    )
    comment = openrouter_response.choices[0].message.content
    print(comment)

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "situation": entry["situation"],
        "situation_json": entry["situation_json"],
        "events": events,
        "comment": comment,
    })
    
    return results


situations = load_dataset("lucaelin/domain_situations_v1", split="train")
situational_event_commentary = situations.map(
    sane_batch_map(generate_event_dialog, {"profession": [], "process": [], "situation": [], "situation_json": [], "events": [], "comment": []}), 
    batched=True, 
    batch_size=1, 
    num_proc=1, 
    remove_columns=situations.column_names
)
situational_event_commentary.sort(["profession", "process"]).to_list()
#situational_event_commentary.sort(["profession", "process"]).push_to_hub("lucaelin/generic_situational_event_commentary_v1")

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

ChatCompletion(id='gen-1733600989-LicEnOuhGZgxuWzMMOsS', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='xqT7fF3Jw', function=Function(arguments='{"event_name": "Invoice generated"}', name='select_event_name'), type='function', index=0)]))], created=1733600989, model='mistralai/mixtral-8x22b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=499, total_tokens=526, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')
selected Invoice generated
available functions ['Account aged', 'Account marked for collection', 'Customer credit check passed', 'Customer credit limit exceeded', 'Customer disputes charge', 'Debt written off', 'Invoice adjusted', 'Invoice generated', 'Payment plan set up', 'Payment receiv

Map:  10%|█         | 1/10 [00:11<01:44, 11.63s/ examples]

 A new invoice was created, followed by a successful credit check for Jane Doe; a payment of $1250.5 has been received, an invoice was adjusted with a deduction of $50, and a credit check for John Smith has passed, all in the midst of ongoing financial tasks and communication with the finance manager.
ChatCompletion(id='gen-1733601001-iFvw6g4QnQGSQc8AkjF8', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='OObMEKb0u', function=Function(arguments='{"event_name": "Budget created"}', name='select_event_name'), type='function', index=0)]))], created=1733601001, model='mistralai/mistral-nemo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=377, total_tokens=399, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')


Map:  20%|██        | 2/10 [00:20<01:18,  9.84s/ examples]

...

Map:  90%|█████████ | 9/10 [01:30<00:09,  9.40s/ examples]

'Project Proposal, after Jane Smith's proofreading and minor edits, has been finalized and approved by John Doe on June 15, 2024.'
ChatCompletion(id='gen-1733601079-hd3lDgaVZbIjfqPCGuYl', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='grD6l876M', function=Function(arguments='{"event_name": "Email Marked as Read"}', name='select_event_name'), type='function', index=0)]))], created=1733601079, model='mistralai/mistral-large', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=515, total_tokens=544, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')
selected Email Marked as Read
available functions ['Email Archived', 'Email Deleted', 'Email Flagged', 'Email Forwarded', 'Email Labeled', 'Email Marked as Read', 'E

Map: 100%|██████████| 10/10 [01:43<00:00, 10.38s/ examples]

You marked an email as read and subsequently archived it, while another email from a previous month was also marked as read.


[{'profession': 'Accountant',
  'process': 'Accounts Receivable',
  'situation': 'In a bustling yet quiet office, an Accountant sits at a sturdy mahogany desk, immersed in the Accounts Receivable process. They are surrounded by a plethora of tools and equipment, all essential to their role. On their right, a reliable laptop - its silver exterior glinting under the warm office lights - is open, displaying the latest version of a sophisticated accounting software, its colorful pie charts and graphs a testament to its efficiency. To their left, a calculator sits alongside a high-quality printer, both in pristine condition. The accountant is not alone; their colleagues from the finance team are scattered around the room, each absorbed in their own tasks, the soft tapping of keys providing a rhythmic backdrop to the office. As the accountant ensures that customers pay their outstanding invoices on time and maintains accurate financial records, they are also in close communication with the f